# Análisis Exploratorio, Preparación y ML (Colab Ready)

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip -q install pandas numpy seaborn matplotlib scikit-learn scipy


In [ ]:
# (Opcional) Subir data/tamizajes.csv en Colab
uploaded = {}
if IN_COLAB:
    from google.colab import files
    print('Ejecuta: uploaded = files.upload() y selecciona data/tamizajes.csv')


In [ ]:
import os, warnings
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
sns.set_palette('husl'); plt.style.use('seaborn-v0_8-darkgrid')
colors=['#2E86AB','#E63946','#F18F01']


## Cargar datos (sin limpiar) y EDA

In [ ]:
candidates=['data/tamizajes.csv','tamizajes.csv']
if 'uploaded' in globals() and isinstance(uploaded,dict) and len(uploaded): candidates=[list(uploaded.keys())[0]]+candidates
data_path=next((p for p in candidates if p and os.path.exists(p)), None)
if not data_path: raise FileNotFoundError('Sube data/tamizajes.csv o ajusta la ruta')
print('Usando archivo:', data_path)
df=pd.read_csv(data_path, sep=';', encoding='latin1')
df.head()


In [ ]:
# Gráfico 1: Distribución por Grupo de Tamizaje
grupo_counts=df['GrupoTamizaje'].value_counts(); grupo_pct=(grupo_counts/len(df))*100
plt.figure(figsize=(10,6)); bars=plt.bar(range(len(grupo_counts)), grupo_counts.values, color=colors[:len(grupo_counts)], alpha=0.8, edgecolor='black', linewidth=2)
plt.xticks(range(len(grupo_counts)), grupo_counts.index, rotation=25, ha='right', fontsize=11); plt.ylabel('Número de Registros', fontsize=12)
plt.title('Distribución de Registros por Grupo de Tamizaje', fontsize=14, fontweight='bold', pad=15); plt.grid(axis='y', alpha=0.3)
for i,b in enumerate(bars): h=b.get_height(); plt.text(b.get_x()+b.get_width()/2, h+(h*0.01), f'{grupo_counts.iloc[i]:,}\n({grupo_pct.iloc[i]:.1f}%)', ha='center', va='bottom', fontsize=10, fontweight='bold')
plt.tight_layout(); plt.savefig('EDA_01_distribucion_grupo_tamizaje.png', dpi=300, bbox_inches='tight'); plt.show()


In [ ]:
# Gráfico 2: Suma Total de Casos por Grupo
print('
📊 Suma Total de Casos por GrupoTamizaje')
suma_total=df.groupby('GrupoTamizaje')['Casos'].sum().sort_values(ascending=False)
plt.figure(figsize=(10,6)); bars=plt.bar(range(len(suma_total)), suma_total.values, color=colors[:len(suma_total)], alpha=0.85, edgecolor='black', linewidth=2)
plt.xticks(range(len(suma_total)), suma_total.index, rotation=15, ha='right'); plt.ylabel('Total de Casos', fontsize=12)
plt.title('Suma Total de Casos por GrupoTamizaje', fontsize=14, fontweight='bold', pad=15); plt.grid(axis='y', alpha=0.3)
for bar,val in zip(bars, suma_total.values): plt.text(bar.get_x()+bar.get_width()/2, val, f'{val:,.0f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
plt.tight_layout(); plt.savefig('EDA_02_suma_total_casos.png', dpi=300, bbox_inches='tight'); plt.show()


In [ ]:
# Gráfico 3: Heatmap Tipo de Tamizaje vs Grupo
print('
📊 Heatmap: Casos por Tipo de Tamizaje y Grupo')
top_tipos=df.groupby('DetalleTamizaje')['Casos'].sum().nlargest(10).index
df_top_tipos=df[df['DetalleTamizaje'].isin(top_tipos)]
pivot_tipos=df_top_tipos.pivot_table(values='Casos', index='DetalleTamizaje', columns='GrupoTamizaje', aggfunc='sum', fill_value=0)
pivot_tipos=pivot_tipos.loc[top_tipos]
plt.figure(figsize=(10,6)); sns.heatmap(pivot_tipos.T, annot=False, cmap='YlOrRd', linewidths=1, linecolor='white', cbar_kws={'label':'Total de Casos'})
plt.xlabel('Tipo de Tamizaje'); plt.ylabel('Grupo Tamizaje'); plt.title('Heatmap: Casos por Tipo de Tamizaje y Grupo', fontsize=14, fontweight='bold', pad=15)
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.savefig('EDA_03_heatmap_tipo_grupo.png', dpi=300, bbox_inches='tight'); plt.show()


In [ ]:
# Gráfico 4: Heatmap Departamento vs Grupo
print('
📊 Heatmap: Casos por Departamento y Grupo')
top_depts=df.groupby('Departamento')['Casos'].sum().nlargest(15).index
df_top_dept=df[df['Departamento'].isin(top_depts)]
pivot_dept=df_top_dept.pivot_table(values='Casos', index='Departamento', columns='GrupoTamizaje', aggfunc='sum', fill_value=0)
pivot_dept=pivot_dept.loc[top_depts]
plt.figure(figsize=(10,6)); sns.heatmap(pivot_dept.T, annot=False, cmap='Blues', linewidths=1, linecolor='white', cbar_kws={'label':'Total de Casos'})
plt.xlabel('Departamento'); plt.ylabel('Grupo Tamizaje'); plt.title('Heatmap: Casos por Departamento y Grupo', fontsize=14, fontweight='bold', pad=15)
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.savefig('EDA_04_heatmap_departamento.png', dpi=300, bbox_inches='tight'); plt.show()


## Preparación de datos: tasa de positividad

In [ ]:
# Convertir 'Casos' a numérico si es necesario
if df['Casos'].dtype=='object':
    df['Casos']=pd.to_numeric(df['Casos'].astype(str).str.replace(',','').str.replace(' ',''), errors='coerce').fillna(0)
# Separar TOTAL y POSITIVOS
df_total=df[df['GrupoTamizaje'].str.contains('TOTAL', case=False, na=False)].copy()
df_pos=df[df['GrupoTamizaje'].str.contains('POSITIVOS', case=False, na=False)].copy()
# Renombrar y unir
df_total=df_total.rename(columns={'Casos':'Total'}).drop(columns=['GrupoTamizaje'])
df_pos=df_pos.rename(columns={'Casos':'Positivos'}).drop(columns=['GrupoTamizaje'])
keys=['Anio','NroMes','ubigeo','Departamento','Provincia','Distrito','Sexo','Etapa','DetalleTamizaje']
df_pivot=df_total.merge(df_pos[keys+['Positivos']], on=keys, how='left')
df_pivot['Positivos']=df_pivot['Positivos'].fillna(0)
df_pivot['Tasa_Positividad']=np.where(df_pivot['Total']>0,(df_pivot['Positivos']/df_pivot['Total'])*100,0)
df_pivot=df_pivot[df_pivot['Total']>0].copy()
print('Filas pivot:', len(df_pivot), ' — Tasa promedio:', round(df_pivot['Tasa_Positividad'].mean(),2),'%')


## Limpieza de datos

In [ ]:
before=len(df_pivot); before_avg=df_pivot['Tasa_Positividad'].mean()
df_clean=df_pivot[df_pivot['Tasa_Positividad']<=100].copy()
after=len(df_clean); after_avg=df_clean['Tasa_Positividad'].mean()
print(f'Eliminados {before-after} | Antes: {before} (avg {before_avg:.2f}%) | Después: {after} (avg {after_avg:.2f}%)')


## Ingeniería de características (one-hot encoding)

In [ ]:
df_encoded=pd.get_dummies(df_clean, columns=['Departamento','Sexo','DetalleTamizaje','Etapa'], dtype=int)
# Eliminar columnas menos útiles
df_encoded=df_encoded.drop(columns=['Provincia','Distrito'])
y=df_encoded['Tasa_Positividad']
X=df_encoded.drop(columns=['Tasa_Positividad','Total','Positivos'])
print('X:', X.shape, ' y:', y.shape)


## Balanceo de datos (submuestreo ceros + KNN oversampling positivos)

In [ ]:
from sklearn.neighbors import NearestNeighbors
zero=(y==0)
X_zero,y_zero=X[zero],y[zero]; X_non_zero,y_non_zero=X[~zero],y[~zero]
print('Original -> ceros:', len(y_zero), '| positivos:', len(y_non_zero))
zero_ratio=0.3; oversample_factor=2
n_zeros_keep=int(len(y_non_zero)/(1-zero_ratio)*zero_ratio)
if n_zeros_keep < len(y_zero):
    idx=np.random.choice(len(y_zero), n_zeros_keep, replace=False); X_zero_s=X_zero.iloc[idx]; y_zero_s=y_zero.iloc[idx]
else:
    X_zero_s,y_zero_s=X_zero,y_zero
print('Ceros conservados:', len(y_zero_s))
X_syn=[]; y_syn=[]
if len(X_non_zero)>1:
    k=min(5, len(X_non_zero)-1); knn=NearestNeighbors(n_neighbors=k+1).fit(X_non_zero)
    n_syn=int(len(X_non_zero)*(oversample_factor-1)); np.random.seed(42)
    for _ in range(n_syn):
        i=np.random.randint(0,len(X_non_zero)); x=X_non_zero.iloc[i].values; yv=y_non_zero.iloc[i]
        neigh=knn.kneighbors([x], return_distance=False)[0]; j=np.random.choice(neigh[1:])
        a=np.random.random(); xn=X_non_zero.iloc[j].values; yn=y_non_zero.iloc[j]
        X_syn.append(x+a*(xn-x)); y_syn.append(yv+a*(yn-yv))
Xb=pd.concat([X_zero_s, X_non_zero, pd.DataFrame(X_syn, columns=X.columns)], ignore_index=True)
yb=pd.concat([y_zero_s, y_non_zero, pd.Series(y_syn, name='Tasa_Positividad')], ignore_index=True)
print('Balanceado -> ceros:', (yb==0).sum(), '| positivos:', (yb>0).sum())
# Guardar y gráfico comparativo
df_balanced=Xb.copy(); df_balanced['Tasa_Positividad']=yb.values
df_balanced.to_csv('dataset_balanceado.csv', index=False, encoding='utf-8-sig')
fig,ax=plt.subplots(1,2,figsize=(14,5))
ax[0].bar(['Ceros','Positivos'], [(yb==0).sum(), (yb>0).sum()], color=['coral','lightblue'], edgecolor='black'); ax[0].set_title('Dataset Balanceado'); ax[0].grid(axis='y', alpha=0.3)
ax[1].bar(['Ceros','Positivos'], [(y==0).sum(), (y>0).sum()], color=['lightgreen','lightblue'], edgecolor='black'); ax[1].set_title('Dataset Original'); ax[1].grid(axis='y', alpha=0.3)
plt.tight_layout(); plt.savefig('DATA_PREP_balanceo_comparacion.png', dpi=300, bbox_inches='tight'); plt.show()


## Modelo: split, entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
MODEL_TYPE='gradient_boosting'  # 'random_forest' o 'gradient_boosting'
X_train,X_test,y_train,y_test=train_test_split(Xb,yb,test_size=0.2,random_state=42)
if MODEL_TYPE=='random_forest':
    model=RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, oob_score=True)
else:
    model=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train,y_train)
y_pred_tr=model.predict(X_train); y_pred_te=model.predict(X_test)
r2_tr=r2_score(y_train,y_pred_tr); r2_te=r2_score(y_test,y_pred_te)
mae_tr=mean_absolute_error(y_train,y_pred_tr); mae_te=mean_absolute_error(y_test,y_pred_te)
rmse_tr=np.sqrt(mean_squared_error(y_train,y_pred_tr)); rmse_te=np.sqrt(mean_squared_error(y_test,y_pred_te))
print(f'Train -> R2:{r2_tr:.4f} MAE:{mae_tr:.4f}% RMSE:{rmse_tr:.4f}%')
print(f'Test  -> R2:{r2_te:.4f} MAE:{mae_te:.4f}% RMSE:{rmse_te:.4f}%')


## Evaluación y gráficos

In [ ]:
import os
os.makedirs('docs', exist_ok=True)
fig,ax=plt.subplots(1,2,figsize=(16,6))
ax[0].scatter(y_train,y_pred_tr,alpha=0.5,s=10,color='steelblue'); ax[0].plot([y_train.min(),y_train.max()],[y_train.min(),y_train.max()],'r--',lw=2); ax[0].set_title(f'Train (R²={r2_tr:.3f})'); ax[0].grid(alpha=0.3); ax[0].set_xlabel('Reales'); ax[0].set_ylabel('Pred')
ax[1].scatter(y_test,y_pred_te,alpha=0.5,s=10,color='darkorange'); ax[1].plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()],'r--',lw=2); ax[1].set_title(f'Test (R²={r2_te:.3f})'); ax[1].grid(alpha=0.3); ax[1].set_xlabel('Reales'); ax[1].set_ylabel('Pred')
plt.tight_layout(); plt.savefig('docs/evaluation_actual_vs_predicted.png', dpi=300, bbox_inches='tight'); plt.show()
imp=getattr(model,'feature_importances_', None)
if imp is not None:
    importances=pd.DataFrame({'Feature':Xb.columns,'Importance':imp}).sort_values('Importance',ascending=False).head(15)
    plt.figure(figsize=(10,8)); plt.barh(range(len(importances)), importances['Importance'], color='darkorange'); plt.yticks(range(len(importances)), importances['Feature']); plt.gca().invert_yaxis(); plt.tight_layout(); plt.savefig('docs/evaluation_feature_importance.png', dpi=300, bbox_inches='tight'); plt.show()
else:
    print('El modelo no expone feature_importances_')


## Curva de aprendizaje (Gradient Boosting)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
if isinstance(model, GradientBoostingRegressor):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    if hasattr(model,'train_score_'):
        plt.plot(model.train_score_, color='steelblue'); plt.title('Score de Entrenamiento'); plt.grid(alpha=0.3)
    else:
        plt.text(0.5,0.5,'train_score_ no disponible', ha='center')
    plt.subplot(1,2,2)
    cum=np.cumsum(sorted(model.feature_importances_, reverse=True))
    plt.plot(cum, color='darkorange'); plt.axhline(y=0.95, color='r', ls='--'); plt.title('Importancia Acumulada'); plt.grid(alpha=0.3)
    plt.tight_layout(); plt.savefig('EVALUATION_learning_curve_GB.png', dpi=300, bbox_inches='tight'); plt.show()
else:
    print('Aplica sólo a Gradient Boosting')


## Guardar modelo

In [ ]:
import pickle
with open('trained_model.pkl','wb') as f:
    pickle.dump({'model':model,'feature_names':list(Xb.columns),'model_type':MODEL_TYPE,'metrics':{'train':{'R2':r2_tr,'MAE':mae_tr,'RMSE':rmse_tr},'test':{'R2':r2_te,'MAE':mae_te,'RMSE':rmse_te}}}, f)
print('Modelo guardado en trained_model.pkl')
